In [1]:
from sage.all import *
from sage.rings.number_field.number_field import CyclotomicField
from sage.rings.integer_ring import ZZ
from sage.modules.free_module_element import vector
from sage.numerical.mip import MixedIntegerLinearProgram as MILP

import random

In [2]:
class Environment:
    
    def __init__(self, p):
        assert p.is_prime()
        self.p = p
        
        # Create K_p
        KK.<z> = CyclotomicField(p)
        self.K = KK
        self.z = z
        self.G = self.K.galois_group()
        
        # Create K_p^+ and \\tau
        (s,) = self.G.gens()
        sigma_neg_one = s ^ (p // 2)
        H = self.G.subgroup([sigma_neg_one])
        self.Kp, self.f = H.fixed_field()
        (self.tau,) = self.Kp.galois_group().gens()
        
        # Solve for c and d
        t = self.theta(1)
        for c in range(1, p):
            if self.f(self.tau(t)) == self.theta(c):
                break
        self.c = ZZ(c)
        self.d = ZZ(mod(c, p)^(-1))
        
        # Create \\Xi
        self.m = (self.p - 1) // 2
        self.etas = [self.eta(ell) for ell in range(1, self.m)]
        self.Xi = matrix([
            [log(abs(self.conj(e, i))) for e in self.etas]
            for i in range(1, self.m)
        ])
        
    def print(self):
        print(f"\n\nEnvironment(p={self.p}) ==========")
        print("\nK:", self.K)
        print("\nz:", self.z)
        print("\nG:", self.G)
        print("\nKp:", self.Kp)
        print("\nc:", self.c)
        print("\nd:", self.d)
        print("\ntau:", self.tau)
        print("\nm:", self.m)
        print("\netas:", self.etas)
        print("\nXi:\n", self.Xi, sep="")
        print("\n==========\n\n")

    def theta(self, j):
        """Calculate \\theta_j"""
        return self.z^j + self.z^(self.p-j)

    def eta(self, ell):
        """Calculate \\eta_\\ell"""
        return self.z^ell * (1 - self.z) / (1 - self.z ^ (2 * ell + 1))

    def conj(self, a, i):
        """Calculate a^{(i)}"""
        return self.f((self.tau ^ i)(a))

    def maxconj(self, a):
        """Calculate \\maxconj(a)"""
        conjs = [self.conj(a, i) for i in range(self.m)]
        abs_conjs = [ZZ(sgn(real(c))) * c for c in conjs]
        i, a = max(enumerate([real(a) for a in abs_conjs]), key=lambda x: x[1])
        return i+1, abs_conjs[i]
    
    def Xi_i(self, i):
        """Calculate \\Xi_i"""
        row_i = self.Xi[i]
        A = matrix([row_i] * self.Xi.nrows())
        A[i] = 0
        return self.Xi - A

    def eps_i(self, M, i):
        """Calculate \\varepsilon_i \\in E'_M"""

        # Create \\Xi_i and L_i for ILP
        X_i = self.Xi_i(i)
        L_i = vector([-log(self.m-2)]*(self.m-1), RR)
        L_i[i] = log(M/2)

        # Solve Linear Programming Problem
        ilp = MILP()
        k_var = ilp.new_variable(integer=True, nonnegative=False)
        ks = [k_var[f"k{j}"] for j in range(1,self.m)]
        objective = sum(k * log(abs(e)) for (k,e) in zip(ks, self.etas))
        ilp.set_objective(-objective)
        for x, l in zip(X_i, L_i):
            c = sum(a * k for (a,k) in zip(x, ks)) <= l
            ilp.add_constraint(c)
        ilp.solve()

        # Recover \\varepsilon_i from ILP solution
        ks_solved = [ilp.get_values(k, convert=ZZ, tolerance=0.0001) for k in ks]
        epsilon = prod(e ^ k for (k,e) in zip(ks_solved, self.etas))

        return epsilon

    def E_prime(self, M):
        """Calculate E'_M"""
        return [self.eps_i(M, i) for i in range(self.m-1)]

    def w(self, i):
        return ZZ((self.d^i * (self.p-1)//2) % self.p)

    def E(self, M):
        """Calculate E_M"""
        Ep = self.E_prime(M)
        ts = [self.theta(self.w(k)) for k in range(1, self.m)]
        conj_signs = [ZZ(sgn(real(self.conj(e, k)))) for k, e in enumerate(Ep, start=1)]
        eps = [s * e for s, e in zip(conj_signs, Ep)]
        return eps + [t * e for (t,e) in zip(ts, eps)]
    
    def make_alg(self, M):
        Es = [self.E(2**k) for k in range(int(log(M, 2))+1)]
        print("Es:")
        for (k, E_) in zip(range(int(log(M, 2))+1), Es):
            print(k,":",max([round(float(real(e)), 3) for e in E_]), [round(float(real(e)), 3) for e in E_])

        def alg(omega):
            nonlocal Es
            a = 0
            for k, E in zip(range(int(log(M, 2))+1), Es):
                print("\n===\nE:", k, ":", E)
                print("a:", a, "=", float(real(a)))
                print("===\n")
                while real(a - omega) * (-1)^k < 0:
                    print("---")
                    ell, _ = self.maxconj(a)
                    al = self.conj(a, ell)
                    ell_ = ell - 1
                    e = (-1)**k * ZZ(sgn(real(al)))
                    print("sgn(a^(ell)):", e)
                    print("ell:", ell)
                    if e == -1:
                        ep = E[ell_]
                    else:
                        ep = E[ell_+self.m-2]
                    if real(ep) < 0:
                        ep *= -1
                    a += (-1)**k * ep
                    print("ep:", ep, "=", float(real(ep)))
                    print("a:", float(real(a)), "=", a)
                    print("---")
            return a
        
        def alg_no_print(omega):
            nonlocal Es
            a = 0
            for k, E in zip(range(int(log(M, 2))+1), Es):
                while real(a - omega) * (-1)^k < 0:
                    ell, _ = self.maxconj(a)
                    al = self.conj(a, ell)
                    ell_ = ell - 1
                    e = (-1)**k * ZZ(sgn(real(al)))
                    if e == -1:
                        ep = E[ell_]
                    else:
                        ep = E[ell_+self.m-2]
                    if real(ep) < 0:
                        ep *= -1
                    a += (-1)**k * ep
            return a
        
        def alt(omega):
            nonlocal Es
            a = 0
            for k, E in zip(range(int(log(M, 2))+1), Es):
                print("\n===\nE:", k, ":", E)
                print("a:", a, "=", float(real(a)))
                print("===\n")
                while real(a - omega) < 0:
                    print("---")
                    ell, _ = self.maxconj(a)
                    al = self.conj(a, ell)
                    ell_ = ell - 1
                    e = ZZ(sgn(real(al)))
                    print("sgn(a^(ell)):", e)
                    print("ell:", ell)
                    if e == -1:
                        ep = E[ell_]
                    else:
                        ep = E[ell_+self.m-2]
                    if real(ep) < 0:
                        ep *= -1
                    candidate = a + ep
                    print("ep:", ep, "=", float(real(ep)))
                    print("candidate:", float(real(candidate)), "=", candidate)
                    if real(candidate - omega) > 0:
                        print("break\n---")
                        break
                    else:
                        a = candidate
                    print("a:", float(real(a)), "=", a)
                    print("---")
            return a
        
        def alt_no_print(omega):
            nonlocal Es
            a = 0
            for k, E in zip(range(int(log(M, 2))+1), Es):
                while real(a - omega) < 0:
                    ell, _ = self.maxconj(a)
                    al = self.conj(a, ell)
                    ell_ = ell - 1
                    e = ZZ(sgn(real(al)))
                    if e == -1:
                        ep = E[ell_]
                    else:
                        ep = E[ell_+self.m-2]
                    if real(ep) < 0:
                        ep *= -1
                    candidate = a + ep
                    if real(candidate - omega) > 0:
                        break
                    else:
                        a = candidate
            return a

        return alg, alt, alg_no_print, alt_no_print

In [3]:
env = Environment(p=5)
env.print()
alg, alt, alg_no_print, alt_no_print = env.make_alg(M=128)



Environment(p=5) ==========

K: Cyclotomic Field of order 5 and degree 4

z: z

G: Galois group 4T1 (4) with order 4 of x^4 + x^3 + x^2 + x + 1

Kp: Number Field in z0 with defining polynomial x^2 - x - 1 with z0 = -0.618033988749895?

c: 2

d: 3

tau: (1,2)

m: 2

etas: [-z^3 - z^2 - 1]

Xi:
[0.481211825059603]



Es:
0 : 2.618 [2.618, 1.618]
1 : 1.0 [1.0, 0.618]
2 : -0.382 [-0.618, -0.382]
3 : 0.382 [0.382, 0.236]
4 : 0.146 [0.146, 0.09]
5 : -0.056 [-0.09, -0.056]
6 : -0.021 [-0.034, -0.021]
7 : 0.021 [0.021, 0.013]


In [6]:
# Random Test
coeff = random.randint(-20, 20)
exponent = random.randint(-20, 20)
offset = -1 * floor(real(coeff * (env.z^exponent + env.z^(-exponent))))
original = coeff * env.z^exponent + coeff * env.z^(-exponent) + offset
omega = real(original)
print("z:", complex(env.z), "=", complex(exp(2 * pi * I / env.p)))
print("omega:", float(omega))
# a = alt_no_print(omega)
# a = alt(omega)
a = alg_no_print(omega)
print("="*10)
print("original:", float(real(original)), "=", original)
print("approx:  ", float(real(a)), "=", a)
print("diff:", float(real(omega - a)))

z: (0.30901699437494745+0.9510565162951535j) = (0.30901699437494745+0.9510565162951535j)
omega: 0.49342219125178843
original: 0.49342219125178843 = 17*z^3 + 17*z^2 + 28
approx:   0.49342219125178843 = 17*z^3 + 17*z^2 + 28
diff: 0.0


In [5]:
# omega = 0.3
original_11 = 18 * env.z**2 + 18 * env.z**(-2) - 14
original_7 = 18 * env.z**2 + 18 * env.z**(-2) + 9
original_5 = 18 * env.z**2 + 18 * env.z**(-2) + 30
original = original_5
omega = real(original)
print("z:", complex(env.z), "=", complex(exp(2 * pi * I / env.p)))
print("omega:", float(omega))
a = alt(omega)
print("="*10)
print("original:", float(real(original)), "=", original)
print("approx:  ", float(real(a)), "=", a)
print("diff:", float(real(omega - a)))

z: (0.30901699437494745+0.9510565162951535j) = (0.30901699437494745+0.9510565162951535j)
omega: 0.8753882025018918

===
E: 0 : [-z^3 - z^2 + 1, -z^3 - z^2]
a: 0 = 0.0
===

---
sgn(a^(ell)): 0
ell: 1
ep: -z^3 - z^2 + 1 = 2.618033988749895
candidate: 2.618033988749895 = -z^3 - z^2 + 1
break
---

===
E: 1 : [1, -z^3 - z^2 - 1]
a: 0 = 0.0
===

---
sgn(a^(ell)): 0
ell: 1
ep: 1 = 1.0
candidate: 1.0 = 1
break
---

===
E: 2 : [z^3 + z^2 + 1, -z^3 - z^2 - 2]
a: 0 = 0.0
===

---
sgn(a^(ell)): 0
ell: 1
ep: -z^3 - z^2 - 1 = 0.6180339887498949
candidate: 0.6180339887498949 = -z^3 - z^2 - 1
a: 0.6180339887498949 = -z^3 - z^2 - 1
---
---
sgn(a^(ell)): 1
ell: 2
ep: z^3 + z^2 + 2 = 0.3819660112501051
candidate: 1.0 = 1
break
---

===
E: 3 : [z^3 + z^2 + 2, -2*z^3 - 2*z^2 - 3]
a: -z^3 - z^2 - 1 = 0.6180339887498949
===

---
sgn(a^(ell)): 1
ell: 2
ep: -2*z^3 - 2*z^2 - 3 = 0.2360679774997898
candidate: 0.8541019662496847 = -3*z^3 - 3*z^2 - 4
a: 0.8541019662496847 = -3*z^3 - 3*z^2 - 4
---
---
sgn(a^(ell)):